In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [2]:
# MNIST Dataset
num_classes = 10
num_features = 784

learning_rate = 0.001
training_steps = 2000
batch_size = 64
display_step = 100

num_input = 28
timesteps = 28
num_units = 32

In [3]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, 28, 28]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
class BiRNN(Model):
    def __init__(self):
        super(BiRNN, self).__init__()
        lstm_fw = layers.LSTM(units = num_units)
        lstm_bw = layers.LSTM(units = num_units, go_backwards = True)
        
        self.bi_lstm = layers.Bidirectional(lstm_fw, backward_layer = lstm_bw)
        self.out = layers.Dense(num_classes)
    
    def call(self, x, is_training = False):
        x = self.bi_lstm(x)
        x = self.out(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x

In [6]:
birnn_net = BiRNN()

In [7]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = x)
    return tf.reduce_mean(loss)

def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis = -1)

In [8]:
optimizer = tf.optimizers.Adam(learning_rate)

In [9]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = birnn_net(x, is_training = True)
        loss = cross_entropy_loss(pred, y)
        
    trainable_variables = birnn_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [10]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = birnn_net(batch_x, is_training = True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("Step: %i, Loss: %f, accuracy: %f" % (step, loss, acc))

Step: 100, Loss: 1.165864, accuracy: 0.718750
Step: 200, Loss: 0.660612, accuracy: 0.843750
Step: 300, Loss: 0.387928, accuracy: 0.921875
Step: 400, Loss: 0.372979, accuracy: 0.921875
Step: 500, Loss: 0.250394, accuracy: 0.937500
Step: 600, Loss: 0.347446, accuracy: 0.890625
Step: 700, Loss: 0.437862, accuracy: 0.875000
Step: 800, Loss: 0.112292, accuracy: 0.968750
Step: 900, Loss: 0.352415, accuracy: 0.890625
Step: 1000, Loss: 0.193233, accuracy: 0.921875
Step: 1100, Loss: 0.143921, accuracy: 0.968750
Step: 1200, Loss: 0.161363, accuracy: 0.953125
Step: 1300, Loss: 0.068597, accuracy: 0.984375
Step: 1400, Loss: 0.194060, accuracy: 0.921875
Step: 1500, Loss: 0.171698, accuracy: 0.984375
Step: 1600, Loss: 0.129564, accuracy: 0.953125
Step: 1700, Loss: 0.218813, accuracy: 0.921875
Step: 1800, Loss: 0.164069, accuracy: 0.953125
Step: 1900, Loss: 0.205504, accuracy: 0.937500
Step: 2000, Loss: 0.109986, accuracy: 0.984375
